# **Diagnóstico de Problemas Veiculares com SBERT e Fuzzy Matching**

Este notebook utiliza o modelo SBERT (Sentence-BERT) para capturar a similaridade semântica entre problemas relatados pelo usuário e problemas registrados em um dataset de diagnósticos veiculares.


In [ ]:
# Importar bibliotecas necessárias
import pandas as pd
import re
import nltk
from sentence_transformers import SentenceTransformer, util
from nltk.corpus import stopwords
import unicodedata

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange




---


##**1. Carregar e Visualizar o Dataset**

### **Importação do dataset**

Aqui vamos importar o dataset para realizar a análise e desenvolver o modelo de IA

In [ ]:
# Carregar o dataset
file_path = '/content/dataset_ia.csv'
df = pd.read_csv(file_path, sep=";")

# Exibir as primeiras linhas do dataset
df.head()


,Fabricante,Modelo,Ano,Problema,Causa,Solucao,Orcamento,Categoria do Problema,Gravidade do Problema,Tempo Estimado de Reparo
0,Chrysler,Eclipse,2020,Ar condicionado não está gelando,Radiador entupido,Ncleo do aquecedor substitudo/lavado/limpo ou ...,1624,Direção,Crítica,4 horas
1,Lexus,Camry,2016,Tremores do motor,Alternador Alternador/Regulador de voltagem i...,Substituio do compressor de ar,1652,Direção,Média,1 dia
2,Ford,Cruze,2010,necessrio redefinir o ciclo de conduo do moni...,Sistema de admisso de ar restrito Sistema de ...,Mdulo de controle de transmisso (TCM) substitu...,903,Direção,Média,1 dia
3,Buick,Sedona,2020,Freios falhando,A velocidade do motor excedeu 200 RPM acima da...,Troca da bateria,1426,Freios,Média,1 dia
4,Cadillac,Camry,2016,Bateria descarregada,Falha na operao de carga parcial da vlvula de ...,Unidade de estgio final de controle do soprado...,1218,Motor,Média,4 horas


### **Análise Inicial**
Vamos realizar uma análise exploratória inicial para entender melhor os dados com os quais estamos trabalhando.

In [ ]:
# Análise exploratória dos dados
print("Resumo das colunas:")
df.info()

print("\nVerificar valores nulos:")
df.isnull().sum()

Resumo das colunas:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Fabricante                1000 non-null   object
 1   Modelo                    1000 non-null   object
 2   Ano                       1000 non-null   int64 
 3   Problema                  1000 non-null   object
 4   Causa                     1000 non-null   object
 5   Solucao                   1000 non-null   object
 6   Orcamento                 1000 non-null   int64 
 7   Categoria do Problema     1000 non-null   object
 8   Gravidade do Problema     1000 non-null   object
 9   Tempo Estimado de Reparo  1000 non-null   object
dtypes: int64(2), object(8)
memory usage: 78.2+ KB

Verificar valores nulos:


,0
Fabricante,0
Modelo,0
Ano,0
Problema,0
Causa,0
Solucao,0
Orcamento,0
Categoria do Problema,0
Gravidade do Problema,0
Tempo Estimado de Reparo,0


In [ ]:
print("\nEstatísticas descritivas de colunas categoricas:")
df.describe(include="object")


Estatísticas descritivas de colunas categoricas:


,Fabricante,Modelo,Problema,Causa,Solucao,Categoria do Problema,Gravidade do Problema,Tempo Estimado de Reparo
count,1000,1000,1000,1000,1000,1000,1000,1000
unique,25,28,32,32,33,8,4,6
top,Volkswagen,XC60,necessrio redefinir o ciclo de conduo do moni...,Falha no sensor de airbags,Conjunto de mdulo de controle do ventilador de...,Sistema de Escape,Média,4 horas
freq,56,45,43,44,42,142,671,374


In [ ]:
print("\nEstatísticas descritivas de colunas numericas:")
df.describe()


Estatísticas descritivas de colunas numericas:


,Ano,Orcamento
count,1000.000000,1000.000000
mean,2016.360000,1305.917000
std,3.409402,613.646106
min,2010.000000,303.000000
25%,2015.000000,808.000000
50%,2017.000000,1218.000000
75%,2019.000000,1830.000000
max,2021.000000,2497.000000


## **2. Pré-processamento dos Dados**

### **Stopwords**

Configura stopwords em português para o pré-processamento do texto. Ou seja palavras comuns e irrelevantes que geralmente não agregam valor ao significado do texto, como "de", "a", "o", "e", "para"...

In [ ]:
# Baixar e configurar stopwords em português
nltk.download('stopwords')
stop_words = set(stopwords.words('portuguese'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### **Função de Pré-processamento**
Essa função realiza o pré-processamento, que inclui:
- Converter o texto para minúsculas
- Remover caracteres especiais
- Remover acentos
- Remover stopwords

O objetivo é normalizar o texto para melhor correspondência semântica.

In [ ]:
# Função de Pré-processamento
def preprocess_text(text):
    # Converte para minúsculas
    text = text.lower()

    # Remove acentos
    text = ''.join(
        c for c in unicodedata.normalize('NFD', text)
        if unicodedata.category(c) != 'Mn'
    )

    # Remove caracteres especiais
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)

    # Remove stopwords
    words = [word for word in text.split() if word not in stop_words]
    return ' '.join(words)

### **Função para Identificar a Categoria com Base no Problema relatado**

Aqui o objetivo é identificar uma categoria existente no dataset analisando o texto que será testado pelo modelo, isso será utilizado para otimizar o tempo de processamento para encontrar o problema similar.

In [ ]:
# Função para identificar a categoria do problema com base no texto do usuário
def identify_category(user_text, categories):
    normalized_categories = [preprocess_text(cat) for cat in categories]  # Pré-processa as categorias
    user_text = preprocess_text(user_text)  # Pré-processa o texto do usuário
    for category in normalized_categories:
        if category in user_text:  # Verifica se a categoria está presente no texto do usuário
            return category
    return None  # Retorna None se nenhuma categoria for identificada

## **3. Desenvolvimento do Modelo com SBERT**

### **Carregar o Modelo SBERT Multilíngue**

In [ ]:
# Carregar o Modelo SBERT Multilíngue
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


### **Função para Encontrar o Problema Mais Próximo do relatado**
Esta função tem como objetivo comparar o problema descrito pelo usuário com uma lista de problemas conhecidos e encontrar o mais parecido.

Como está funcionando
1. **Preparação do Texto:**

- Primeiro, a função limpa o texto do problema do usuário e dos problemas no dataset (removendo acentos, deixando tudo em minúsculas e retirando palavras comuns).
2. **Transformação em Vetores:**

- A função usa o modelo SBERT para transformar cada descrição de problema em um vetor numérico chamado "embedding". Esses vetores representam o "significado" de cada frase de forma que o computador possa comparar.
3. **Comparação de Similaridade:**

- Depois que os vetores estão prontos, a função calcula o quanto cada problema no dataset é parecido com o problema do usuário. Isso é feito medindo o "ângulo" entre os vetores – quanto menor o ângulo, maior a similaridade.
4. **Filtros de Fabricante, Modelo e Ano (Opcional):**

- Se o usuário fornecer informações extras como o fabricante, modelo ou ano do veículo, a função usa esses dados para refinar ainda mais os resultados, ajudando a encontrar uma correspondência mais específica.
5. **Retorno do Problema Mais Próximo:**

No final, a função retorna o problema do dataset que teve a maior similaridade com o problema do usuário, juntamente com o valor de similaridade.

In [ ]:
# Função para encontrar o problema mais próximo com base na similaridade
def find_similar_problem(user_input, df, model, similarity_threshold=0.7, fabricante=None, modelo=None, ano=None):
    # Identificar a categoria do problema do usuário
    df['Categoria do Problema'] = df['Categoria do Problema'].apply(preprocess_text)
    categories = df['Categoria do Problema'].unique()
    category = identify_category(user_input, categories)

    if category:
        # Filtrar o conjunto de dados para a categoria identificada
        category_df = df[df['Categoria do Problema'] == category].copy()
    else:
        # Se nenhuma categoria for encontrada, considerar todo o dataset
        category_df = df.copy()

    # Pré-processar os problemas da categoria filtrada
    category_df['Problema_Processed'] = category_df['Problema'].apply(preprocess_text)

    # Criar embeddings para os problemas da categoria filtrada
    category_df['Problema_Embedding'] = category_df['Problema_Processed'].apply(lambda x: model.encode(x, convert_to_tensor=True))

    # Criar o embedding para o problema do usuário
    user_embedding = model.encode(preprocess_text(user_input), convert_to_tensor=True)

    # Calcular similaridade entre o problema do usuário e os problemas da categoria
    similarities = []
    for _, row in category_df.iterrows():
        similarity = util.pytorch_cos_sim(user_embedding, row['Problema_Embedding']).item()
        similarities.append((row, similarity))

   # Ordena por similaridade em ordem decrescente
    sorted_similarities = sorted(similarities, key=lambda x: x[1], reverse=True)

    # Filtrar problemas com similaridade acima do limite
    high_similarity_problems = [(row, similarity) for row, similarity in sorted_similarities if similarity >= similarity_threshold]

    if high_similarity_problems:
        # Se múltiplos problemas similares forem encontrados, aplicar filtro adicional por Fabricante, Modelo e Ano
        if fabricante or modelo or ano:
            filtered_problems = [
                (row, similarity) for row, similarity in high_similarity_problems
                   if (modelo is None or row.get('Modelo') == modelo) and
                   (fabricante is None or row.get('Fabricante') == fabricante) and
                   (ano is None or row.get('Ano') == ano)
            ]
            # Se problemas foram filtrados por Fabricante, Modelo e Ano, use o mais similar entre eles
            if filtered_problems:
                best_match_row, best_similarity = max(filtered_problems, key=lambda x: x[1])
                return best_match_row, best_similarity

        # Caso não tenha conseguido filtrar, retorna o mais similar entre os problemas encontrados
        best_match_row, best_similarity = high_similarity_problems[0]
        return best_match_row, best_similarity
    else:
        return None, 0  # Se a similaridade for insuficiente

---
## **4. Teste do modelo**

Aqui testamos o modelo criado passando os dados necessários e esperando que a linha do dataset seja retornada ou que o resultado seja Vazio caso nenhum Problema do dataset seja condizente com o problema relatado pelo usuario.

In [ ]:
# Exemplo de avaliação com um problema do usuário, especificando fabricante, modelo e ano
user_problem = "motor aquecendo"
fabricante = "Honda"
modelo = "Civic"
ano = 2012

best_match_row, similarity = find_similar_problem(user_problem, df, model, similarity_threshold=0.7, fabricante=fabricante, modelo=modelo, ano=ano)

# Exibir o resultado
if best_match_row is not None:
    print(f"Problema do usuário: {user_problem}")
    print(f"Problema mais similar encontrado: {best_match_row['Problema']}")
    print(f"Similaridade: {similarity:.2f}\n")
    print(f"Linha completa do problema encontrado:\n{best_match_row}")
else:
    print(f"Nenhum problema similar encontrado para: {user_problem}")


Problema do usuário: motor aquecendo
Problema mais similar encontrado: Motor superaquecendo
Similaridade: 0.90

Linha completa do problema encontrado:
Fabricante                                                             Subaru
Modelo                                                                 Encore
Ano                                                                      2017
Problema                                                 Motor superaquecendo
Causa                       A velocidade do motor excedeu 200 RPM acima da...
Solucao                     Substituiu o motor de arranque ou consertou a ...
Orcamento                                                                1086
Categoria do Problema                                                   motor
Gravidade do Problema                                                   Média
Tempo Estimado de Reparo                                              4 horas
Problema_Processed                                       motor supera

---
## **4. Conclusão**

Neste projeto, utilizamos o modelo SBERT (Sentence-BERT) para construir um sistema de diagnóstico que identifica problemas veiculares similares com base na descrição de um problema fornecido pelo usuário.

**Pontos a destacar:**

- Dependência de Dados de Treinamento: A precisão depende de um dataset bem estruturado e de alta qualidade.
- Possível Insensibilidade a Pequenas Diferenças Semânticas: Em alguns casos, o modelo pode não distinguir problemas muito específicos se forem semanticamente semelhantes mesmo sendo esse o ponto mais importante encontrado por nós durante a escolha e desenvolvimento do modelo.
- Ampliação do Dataset: Expandir o dataset com mais descrições de problemas específicos para aumentar a cobertura de casos relatados pelo usuario.
- Ajuste Fino do Modelo: Treinar ou ajustar SBERT em um conjunto de dados especializado em terminologia automotiva para melhorar a precisão, com os dados mais fidedignos e assertivos possíveis.

Em resumo, o uso de SBERT neste projeto demonstrou-se eficaz para identificar problemas veiculares similares, oferecendo uma base sólida para um sistema de recomendação de diagnósticos veiculares.